In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Отключаем вывод предупреждений в проекте.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

Считываем датасет в формате 'utf-8' (для русских букв), обрабатываем его с
помощью регулярных выражений (объект re). Оставим в тексте только символы русских
букв и символы пробела:


In [ ]:
import re

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/2.8 лаб/train_data__positive.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text) # убираем все недопустимые символы

Создаем обучающую выборку с помощью инструмента
tf.keras.preprocessing.text.Tokenizer - «умный» парсинг (разложение на составляющие
элементы) входного текста.

In [ ]:
num_characters = 34  # 33 буквы + пробел
input_chars = 6 # для размера входной строки на предсказание

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=num_characters, char_level=True)
#токенизируем на уровне символов
tokenizer.fit_on_texts([text])
#формируем токены на основе частотности в нашем тексте

Выведем полученный словарь символов с порядковым номером на основе
загруженного датасета:


In [ ]:
print(tokenizer.word_index)

{' ': 1, 'о': 2, 'е': 3, 'т': 4, 'и': 5, 'а': 6, 'н': 7, 'с': 8, 'в': 9, 'р': 10, 'м': 11, 'л': 12, 'ь': 13, 'д': 14, 'п': 15, 'у': 16, 'ы': 17, 'з': 18, 'я': 19, 'б': 20, 'ч': 21, 'к': 22, 'й': 23, 'ж': 24, 'г': 25, 'ш': 26, 'х': 27, 'ю': 28, 'ц': 29, 'щ': 30, 'э': 31, 'ф': 32, 'ъ': 33}


Преобразуем текст в набор OHE-векторов.

In [ ]:
data = tokenizer.texts_to_matrix(text)  # преобразуем исходный текст в массив OHE

In [ ]:
print(data.shape)

(6307, 34)


Далее из этой матрицы сформируем тензор обучающей выборки и
соответствующий набор выходных значений.
Для начала вычислим размер обучающего множества:

In [ ]:
n = data.shape[0] - input_chars  # так как мы предсказываем по шести символам - седьмой

Сформируем входной тензор и прогнозные значения:

In [ ]:
import numpy as np

In [ ]:
X = np.array([data[i:i + input_chars, :] for i in range(n)])
Y = data[input_chars:]  # предсказание следующего символа

Разработаем модель рекуррентной НС
Подключаем зависимости в проект, для данной подзадачи

In [ ]:
from keras.models import Sequential
from keras.layers import Input, Dense, SimpleRNN

Разрабатываем модель НС

In [ ]:
model = Sequential()
#при тренировке в рекуррентные модели keras подается сразу
#вся последовательность, поэтому в input два числа: 1-длина последовательности (=6), 2-размер OHE (=34)
model.add(Input((input_chars,num_characters)))
model.add(SimpleRNN(500, activation='tanh'))  # рекуррентный слой на 500 нейронов
model.add(Dense(num_characters, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 500)               267500    
                                                                 
 dense (Dense)               (None, 34)                17034     
                                                                 
Total params: 284534 (1.09 MB)
Trainable params: 284534 (1.09 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


 Компилируем НС.

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
optimizer='adam')

Обучаем НС

In [ ]:
history = model.fit(X, Y, batch_size=32, epochs=100)

Epoch 1/100
197/197 [==============================] - 4s 20ms/step - loss: 0.4758 - accuracy: 0.8627
Epoch 2/100
197/197 [==============================] - 3s 17ms/step - loss: 0.4687 - accuracy: 0.8610
Epoch 3/100
197/197 [==============================] - 3s 17ms/step - loss: 0.4902 - accuracy: 0.8568
Epoch 4/100
197/197 [==============================] - 5s 25ms/step - loss: 0.4622 - accuracy: 0.8608
Epoch 5/100
197/197 [==============================] - 5s 24ms/step - loss: 0.4366 - accuracy: 0.8741
Epoch 6/100
197/197 [==============================] - 3s 17ms/step - loss: 0.4470 - accuracy: 0.8692
Epoch 7/100
197/197 [==============================] - 3s 17ms/step - loss: 0.4388 - accuracy: 0.8718
Epoch 8/100
197/197 [==============================] - 4s 20ms/step - loss: 0.4598 - accuracy: 0.8635
Epoch 9/100
197/197 [==============================] - 5s 27ms/step - loss: 0.4452 - accuracy: 0.8702
Epoch 10/100
197/197 [==============================] - 4s 19ms/step - loss: 0.442

In [ ]:
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


 Разработаем вспомогательную функцию для прогнозирования «следующего»
символа и добавления его в конец начальной строки

In [ ]:
def buildPhrase(input_str, str_len=50):
    for i in range(str_len):
        x = []
        for j in range(i, i + input_chars):
            x.append(tokenizer.texts_to_matrix(input_str[j]))  # преобразуем символы в One-Hot-encoding

        x = np.array(x)
        inp = x.reshape(1, input_chars, num_characters)

        pred = model.predict(inp)  # предсказываем OHE четвертого символа
        d = tokenizer.index_word[pred.argmax(axis=1)[0]]  # получаем ответ в символьном представлении
        input_str += d  # дописываем строку

    return input_str

Эксперименты


1 - Пример из датасета на 6 входных символов «лучший»:

In [ ]:
res = buildPhrase("лучший")
print(res)

1/1 [==============================] - 0s 34ms/step
лучший жизни весну к вашей жизни весну к вашей жизни вес


2 - Пример из датасета на 6 входных символов «думайт»:

In [ ]:
res = buildPhrase("думайт") #6 входных символов
print(res)

1/1 [==============================] - 0s 22ms/step
думайте позитивным мышлением привнести позитивным мышлен


3 - Модификация прошлого примера на 7 входных символов «думайтее»:

In [ ]:
res = buildPhrase("думайтее") #7 входных символов
print(res)

1/1 [==============================] - 0s 22ms/step
думайтеее   нвсостштриооя   тпбооидттеье н вавпссотелр ьуа


4 - Пример из датасета на 6 входных символов «супер »:

In [ ]:
res = buildPhrase("уникальны") #10
print(res)

1/1 [==============================] - 0s 23ms/step
уникальныьны  ы ввздосяммт  оогпззориититвь ьоеп лтриаоиймт


5 - Пример из датасета на 6 входных символов «новый »:

In [ ]:
res = buildPhrase("супер ")  #6
print(res)

1/1 [==============================] - 0s 22ms/step
супер герой твои утренним проблемам помешать вам суждено
